In [1]:
import pandas as pd

DATA_PATH = '/Users/diego/Documents/airbnb_csv/'

df = pd.read_csv(DATA_PATH+'new-york-city-airbnb-open-data/AB_NYC_2019.csv')
df1 = pd.read_csv(DATA_PATH+'New York.csv')
df2 = pd.read_csv(DATA_PATH+'listings_summary.csv')
df3 = pd.read_csv(DATA_PATH+'airbnb_nyc_homes_raw_20180908.csv')

/Users/diego/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (43,87,88) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
# only df and df2 have "interpretable" dates

df['last_review'] = pd.to_datetime(df['last_review'], infer_datetime_format=True)
df2['last_review'] = pd.to_datetime(df2['last_review'], infer_datetime_format=True) # needed to separate dates

In [5]:
### Merging 4 dataframe to get all features

cols_to = list(set(df2.columns) - set(df1.columns))
list(set(df2.columns) - set(df1.columns)) # subtracting columns of df1 from df2
# give me all the columns from df2 that are NOT in df1
# columns that are only in one dataframe (df2) and use this to select a subset of columns in the merge (to avoid dups)

['availability_365',
 'host_id',
 'name',
 'host_name',
 'neighbourhood',
 'calculated_host_listings_count',
 'neighbourhood_group',
 'last_review']

In [9]:
df2[cols_to].columns

Index(['availability_365', 'host_id', 'name', 'host_name', 'neighbourhood',
       'calculated_host_listings_count', 'neighbourhood_group', 'last_review'],
      dtype='object')

In [136]:
new_df_1 = pd.merge(df1, df2[cols_to], left_index=True, right_index=True, how='inner')

print(new_df_1.shape) # includes df2 (so last_review for dates)
print(new_df_1.isnull().sum().sum())
new_df_1.head()

(44317, 39)
101171


,id,host_response_time,host_response_rate,host_is_superhost,host_has_profile_pic,neighbourhood_cleansed,latitude,longitude,is_location_exact,property_type,...,require_guest_profile_picture,reviews_per_month,availability_365,host_id,name,host_name,neighbourhood,calculated_host_listings_count,neighbourhood_group,last_review
0,18461891,NaN,NaN,f,t,Ditmars Steinway,40.774142,-73.916246,t,Apartment,...,f,NaN,0,916092,"Bright, comfortable 1B studio near everything!",Connie Mae,Ditmars Steinway,1,Queens,NaT
1,20702398,within an hour,100%,f,t,City Island,40.849191,-73.786509,f,House,...,f,2.00,169,1457680,Quiet house on City Island,James,City Island,1,Bronx,2017-10-01
2,6627449,within an hour,100%,f,t,City Island,40.849775,-73.786609,t,Apartment,...,f,0.77,363,13886510,Large 1 BDRM in Great location,Arlene,City Island,1,Bronx,2017-09-26
3,19949243,within a few hours,100%,f,t,City Island,40.848838,-73.782276,f,Boat,...,f,NaN,90,1149260,Stay aboard a sailboat,MoMo,City Island,1,Bronx,NaT
4,1886820,NaN,NaN,f,t,City Island,40.841144,-73.783052,t,House,...,f,NaN,365,9815788,Quaint City Island Community.,Steve,City Island,1,Bronx,NaT


In [11]:
new_df_1.isnull().sum().sort_values(ascending=False) # removing a lot, way too much nans

square_feet                       43768
host_response_time                13679
host_response_rate                13679
review_scores_rating              10099
reviews_per_month                  9474
last_review                        9440
host_is_superhost                   232
host_has_profile_pic                232
host_name                           232
bathrooms                           144
beds                                 91
bedrooms                             73
name                                 28
neighbourhood_cleansed                0
latitude                              0
amenities                             0
is_location_exact                     0
property_type                         0
room_type                             0
accommodates                          0
bed_type                              0
longitude                             0
guests_included                       0
price                                 0
neighbourhood_group                   0


In [14]:
# put in variable
series_new_df_1 = new_df_1.isnull().sum().sort_values(ascending=False)
 
series_new_df_1.index[:5]# remove these (top with nulls) / will remove those with nulls AFTER last_review, manually. important to save last_review (dates)

Index(['square_feet', 'host_response_time', 'host_response_rate',
       'review_scores_rating', 'reviews_per_month'],
      dtype='object')

In [15]:
to_remove_from_df_1 = list(series_new_df_1.index[:5])

In [16]:
new_df_1[to_remove_from_df_1].isnull().sum().sort_values(ascending=False)

square_feet             43768
host_response_rate      13679
host_response_time      13679
review_scores_rating    10099
reviews_per_month        9474
dtype: int64

In [17]:
new_df_1 = new_df_1.drop(columns=to_remove_from_df_1)

In [18]:
# new df with removed excessive nulls

new_df_1.isnull().sum().sort_values(ascending=False)

last_review                       9440
host_is_superhost                  232
host_has_profile_pic               232
host_name                          232
bathrooms                          144
beds                                91
bedrooms                            73
name                                28
room_type                            0
bed_type                             0
accommodates                         0
is_location_exact                    0
property_type                        0
price                                0
longitude                            0
latitude                             0
neighbourhood_cleansed               0
amenities                            0
guests_included                      0
neighbourhood_group                  0
minimum_nights                       0
maximum_nights                       0
calendar_updated                     0
availability_30                      0
number_of_reviews                    0
instant_bookable         

In [19]:
new_df_1.isnull().sum().sum()

10472

## Now other method

In [21]:
new_practice = df1.merge(df2, on='id', how='left')

In [22]:
print(new_practice.shape)
print(new_practice.isnull().sum().sum())
new_practice.head()

(44317, 46)
110645


,id,host_response_time,host_response_rate,host_is_superhost,host_has_profile_pic,neighbourhood_cleansed,latitude_x,longitude_x,is_location_exact,property_type,...,latitude_y,longitude_y,room_type_y,price_y,minimum_nights_y,number_of_reviews_y,last_review,reviews_per_month_y,calculated_host_listings_count,availability_365
0,18461891,NaN,NaN,f,t,Ditmars Steinway,40.774142,-73.916246,t,Apartment,...,40.774142,-73.916246,Entire home/apt,110,6,0,NaT,NaN,1,0
1,20702398,within an hour,100%,f,t,City Island,40.849191,-73.786509,f,House,...,40.849191,-73.786509,Private room,50,1,2,2017-10-01,2.00,1,169
2,6627449,within an hour,100%,f,t,City Island,40.849775,-73.786609,t,Apartment,...,40.849775,-73.786609,Entire home/apt,125,3,21,2017-09-26,0.77,1,363
3,19949243,within a few hours,100%,f,t,City Island,40.848838,-73.782276,f,Boat,...,40.848838,-73.782276,Entire home/apt,100,3,0,NaT,NaN,1,90
4,1886820,NaN,NaN,f,t,City Island,40.841144,-73.783052,t,House,...,40.841144,-73.783052,Entire home/apt,300,7,0,NaT,NaN,1,365


In [23]:
new_practice.isnull().sum().sort_values(ascending=False)

square_feet                       43768
host_response_time                13679
host_response_rate                13679
review_scores_rating              10099
reviews_per_month_y                9474
reviews_per_month_x                9474
last_review                        9440
host_name                           232
host_is_superhost                   232
host_has_profile_pic                232
bathrooms                           144
beds                                 91
bedrooms                             73
name                                 28
bed_type                              0
amenities                             0
availability_365                      0
property_type                         0
accommodates                          0
room_type_x                           0
guests_included                       0
is_location_exact                     0
longitude_x                           0
latitude_x                            0
neighbourhood_cleansed                0


In [24]:
# put in variable
series_new_practice = new_practice.isnull().sum().sort_values(ascending=False)

In [25]:
series_new_practice.index[:6] # remove these (top with nulls) / will remove those with nulls AFTER last_review, manually. important to save last_review (dates)

Index(['square_feet', 'host_response_time', 'host_response_rate',
       'review_scores_rating', 'reviews_per_month_y', 'reviews_per_month_x'],
      dtype='object')

In [26]:
to_remove_from_df_np = list(series_new_practice.index[:6])

In [27]:
new_practice[to_remove_from_df_np].isnull().sum().sort_values(ascending=False)

square_feet             43768
host_response_rate      13679
host_response_time      13679
review_scores_rating    10099
reviews_per_month_x      9474
reviews_per_month_y      9474
dtype: int64

In [28]:
new_practice = new_practice.drop(columns=to_remove_from_df_np)

In [29]:
# new df with removed excessive nulls

new_practice.isnull().sum().sort_values(ascending=False)

last_review                       9440
host_is_superhost                  232
host_has_profile_pic               232
host_name                          232
bathrooms                          144
beds                                91
bedrooms                            73
name                                28
availability_365                     0
price_x                              0
amenities                            0
bed_type                             0
room_type_x                          0
accommodates                         0
minimum_nights_x                     0
property_type                        0
is_location_exact                    0
longitude_x                          0
latitude_x                           0
neighbourhood_cleansed               0
guests_included                      0
calendar_updated                     0
maximum_nights                       0
calculated_host_listings_count       0
number_of_reviews_y                  0
minimum_nights_y         

In [30]:
new_practice.isnull().sum().sum()

10472

### We had over 100,000 nulls in both, now less than 11,000 in both

#### Last steps before comparing types of datasets

In [31]:
new_practice['last_review'] = pd.to_datetime(new_practice['last_review'], infer_datetime_format=True) # needed to separate dates

In [32]:
new_df_1['last_review'] = pd.to_datetime(new_df_1['last_review'], infer_datetime_format=True)

In [35]:
# Main driver of suspicision

new_practice[['latitude_x', 'latitude_y', 'longitude_x', 'longitude_y', 'room_type_x', 'room_type_y']].head(10)

,latitude_x,latitude_y,longitude_x,longitude_y,room_type_x,room_type_y
0,40.774142,40.774142,-73.916246,-73.916246,Entire home/apt,Entire home/apt
1,40.849191,40.849191,-73.786509,-73.786509,Private room,Private room
2,40.849775,40.849775,-73.786609,-73.786609,Entire home/apt,Entire home/apt
3,40.848838,40.848838,-73.782276,-73.782276,Entire home/apt,Entire home/apt
4,40.841144,40.841144,-73.783052,-73.783052,Entire home/apt,Entire home/apt
5,40.850024,40.850024,-73.789328,-73.789328,Private room,Private room
6,40.851980,40.851980,-73.789304,-73.789304,Entire home/apt,Entire home/apt
7,40.852054,40.852054,-73.788680,-73.788680,Private room,Private room
8,40.842124,40.842124,-73.785197,-73.785197,Entire home/apt,Entire home/apt
9,40.853491,40.853491,-73.788607,-73.788607,Private room,Private room


In [39]:
# count amenities, replace row value with #

convert_str_to_list_len = lambda x: len(x.strip('{').strip('}').split(","))
new_df_1['amenities'] = new_df_1['amenities'].apply(convert_str_to_list_len)

In [40]:
# dropping duplicates

dups = ['latitude_x', 'longitude_x', 'room_type_x', 'price_x', 'minimum_nights_x', 'number_of_reviews_x']

In [41]:
new_practice = new_practice.drop(columns=dups)

In [42]:
new_practice.isnull().sum().sort_values(ascending=False)

last_review                       9440
host_name                          232
host_is_superhost                  232
host_has_profile_pic               232
bathrooms                          144
beds                                91
bedrooms                            73
name                                28
guests_included                      0
amenities                            0
bed_type                             0
availability_365                     0
calendar_updated                     0
accommodates                         0
property_type                        0
is_location_exact                    0
neighbourhood_cleansed               0
maximum_nights                       0
instant_bookable                     0
availability_30                      0
calculated_host_listings_count       0
is_business_travel_ready             0
cancellation_policy                  0
require_guest_profile_picture        0
host_id                              0
neighbourhood_group      

In [47]:
to_drop = ['name', 'host_name', 'id', 'host_id', 'host_is_superhost', 'host_has_profile_pic'] 

new_practice = new_practice.drop(columns=to_drop) # on both
new_df_1 = new_df_1.drop(columns=to_drop)

In [48]:
# first on new_practice

In [50]:
new_practice.last_review.value_counts(bins=10) # use specific range of dates

# this is just to get latest dates that also have a lot of values (in bins)

(2017-01-08 09:36:00, 2017-10-02]                       26296
(2016-04-16 19:12:00, 2017-01-08 09:36:00]               5058
(2015-07-25 04:48:00, 2016-04-16 19:12:00]               2542
(2014-10-31 14:24:00, 2015-07-25 04:48:00]                583
(2014-02-07, 2014-10-31 14:24:00]                         244
(2013-05-16 09:36:00, 2014-02-07]                          80
(2012-08-22 19:12:00, 2013-05-16 09:36:00]                 47
(2011-11-30 04:48:00, 2012-08-22 19:12:00]                 15
(2011-03-08 14:24:00, 2011-11-30 04:48:00]                  8
(2010-06-12 08:00:57.599999999, 2011-03-08 14:24:00]        4
Name: last_review, dtype: int64

In [51]:
new_practice = new_practice.copy()
cutoff = pd.to_datetime('2017-01-01')

In [52]:
new_practice_df = new_practice[new_practice['last_review'] >= cutoff] # let's get everything in and above Nov 2017

new_practice_df.shape

(27064, 28)

In [53]:
practice_sorted_df = new_practice_df.sort_values(by='last_review')

practice_sorted_df.head()

,neighbourhood_cleansed,is_location_exact,property_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,guests_included,...,neighbourhood,latitude_y,longitude_y,room_type_y,price_y,minimum_nights_y,number_of_reviews_y,last_review,calculated_host_listings_count,availability_365
23291,Harlem,t,Apartment,2,1.0,0.0,1.0,Real Bed,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",1,...,Harlem,40.814691,-73.939852,Entire home/apt,140,2,15,2017-01-01,1,335
1155,Astoria,t,Apartment,2,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",3,...,Astoria,40.757101,-73.918569,Entire home/apt,110,1,24,2017-01-01,1,6
17389,Flatbush,f,House,2,1.0,1.0,1.0,Real Bed,"{""Wireless Internet"",""Air conditioning"",Kitche...",1,...,Flatbush,40.638357,-73.956975,Private room,85,1,6,2017-01-01,1,0
41210,Williamsburg,t,Apartment,4,1.0,1.0,1.0,Real Bed,"{TV,""Wireless Internet"",""Air conditioning"",Kit...",2,...,Williamsburg,40.718373,-73.955157,Entire home/apt,130,1,1,2017-01-01,1,89
39406,Washington Heights,t,Apartment,4,1.0,1.0,2.0,Real Bed,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,...,Washington Heights,40.856945,-73.934858,Entire home/apt,109,4,12,2017-01-01,1,73


In [54]:
practice_sorted = practice_sorted_df.copy()

def separating_date(df):
    df['month'] = df['last_review'].dt.month
    df['year'] = df['last_review'].dt.year
    df['day'] = df['last_review'].dt.day

separating_date(practice_sorted)

In [55]:
practice_sorted = practice_sorted.dropna(subset=['last_review', 'bedrooms'])

In [59]:
practice_sorted = practice_sorted.drop(columns='last_review')

In [63]:
# now on new_df

In [65]:
new_df_1.last_review.value_counts(bins=10) # use specific range of dates

# this is just to get latest dates that also have a lot of values (in bins)

(2017-01-08 09:36:00, 2017-10-02]                       26296
(2016-04-16 19:12:00, 2017-01-08 09:36:00]               5058
(2015-07-25 04:48:00, 2016-04-16 19:12:00]               2542
(2014-10-31 14:24:00, 2015-07-25 04:48:00]                583
(2014-02-07, 2014-10-31 14:24:00]                         244
(2013-05-16 09:36:00, 2014-02-07]                          80
(2012-08-22 19:12:00, 2013-05-16 09:36:00]                 47
(2011-11-30 04:48:00, 2012-08-22 19:12:00]                 15
(2011-03-08 14:24:00, 2011-11-30 04:48:00]                  8
(2010-06-12 08:00:57.599999999, 2011-03-08 14:24:00]        4
Name: last_review, dtype: int64

In [66]:
new_df_2 = new_df_1.copy()
cutoff = pd.to_datetime('2017-01-01')

In [67]:
new_df_3 = new_df_2[new_df_2['last_review'] >= cutoff] # let's get everything in and above Nov 2017

new_df_3.shape

(27064, 28)

In [68]:
# sorting for time based splits

new_df_3_sorted = new_df_3.sort_values(by='last_review')

new_df_3_sorted.head()

,neighbourhood_cleansed,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,...,number_of_reviews,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,availability_365,neighbourhood,calculated_host_listings_count,neighbourhood_group,last_review
23291,Harlem,40.814691,-73.939852,t,Apartment,Entire home/apt,2,1.0,0.0,1.0,...,15,f,f,strict,f,335,Harlem,1,Manhattan,2017-01-01
1155,Astoria,40.757101,-73.918569,t,Apartment,Entire home/apt,2,1.0,1.0,1.0,...,24,f,f,moderate,f,6,Astoria,1,Queens,2017-01-01
17389,Flatbush,40.638357,-73.956975,f,House,Private room,2,1.0,1.0,1.0,...,6,f,f,strict,f,0,Flatbush,1,Brooklyn,2017-01-01
41210,Williamsburg,40.718373,-73.955157,t,Apartment,Entire home/apt,4,1.0,1.0,1.0,...,1,f,f,strict,f,89,Williamsburg,1,Brooklyn,2017-01-01
39406,Washington Heights,40.856945,-73.934858,t,Apartment,Entire home/apt,4,1.0,1.0,2.0,...,12,f,f,strict,f,73,Washington Heights,1,Manhattan,2017-01-01


In [99]:
### Getting features and target, baseline, (maybe) further feature eng., then Train / Val / Test split

new_df_3_sorted_cop = new_df_3_sorted.copy()

def separating_date(df):
    df['month'] = df['last_review'].dt.month
    df['year'] = df['last_review'].dt.year
    df['day'] = df['last_review'].dt.day

separating_date(new_df_3_sorted_cop)

In [100]:
new_df_3_sorted_cop = new_df_3_sorted_cop.drop(columns='last_review')

## Now, to what I did all of this for

In [104]:
# new_practice

In [105]:
train = practice_sorted[:int((len(practice_sorted) * 80) / 100)] # 23782
train.shape

(21615, 30)

In [106]:
# getting test set (20%)

test = practice_sorted[~practice_sorted.index.isin(train.index)] # 5946 -- # give me the rows (indices) that are NOT in the train index
test.shape

(5404, 30)

In [107]:
# getting val set from train set (same length as test set)

val = train[0:(len(test))]
val.shape

(5404, 30)

In [108]:
# getting new training set, set that is not in val set

new_train = train[~train.index.isin(val.index)] # give me the rows (indices) that are NOT in the val index
new_train.shape

(16211, 30)

In [109]:
# new_df_3_sorted_cop

In [110]:
train_new_df = new_df_3_sorted_cop[:int((len(new_df_3_sorted_cop) * 80) / 100)] # 23782
train_new_df.shape

(21651, 30)

In [111]:
# getting test set (20%)

test_new_df = new_df_3_sorted_cop[~new_df_3_sorted_cop.index.isin(train_new_df.index)] # 5946 -- # give me the rows (indices) that are NOT in the train index
test_new_df.shape

(5413, 30)

In [112]:
# getting val set from train set (same length as test set)

val_new_df = train_new_df[0:(len(test_new_df))]
val_new_df.shape

(5413, 30)

In [113]:
# getting new training set, set that is not in val set

new_train_df = train_new_df[~train_new_df.index.isin(val_new_df.index)] # give me the rows (indices) that are NOT in the val index
new_train_df.shape

(16238, 30)

### Modeling for practice df

In [114]:
# new_practice

In [115]:
target = 'price_y'
features = new_train.drop(columns=target).columns.tolist()

# only when using train test split func
# X = new_train.drop(columns=target)
# y = new_train[target]

x_train = new_train[features]
y_train = new_train[target]

x_val = val[features]
y_val = val[target]

x_test = test[features]
y_test = test[target]

In [118]:
## Modeling

# interesting idea to consider, if I have time: give the user the option to get both 
# a precise, single price (regression, $103) and/or a range/bin of possible prices (class, $90-120)

from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import category_encoders as ce
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler


#### Baseline

# baseline

# guess = y_train.mean()
# y_pred = [guess] * len(y_train)

# print(f'Mean absolute error: {mean_absolute_error(y_train, y_pred):.2f}')
# print(f'Mean squared error: {mean_squared_error(y_train, y_pred):.2f}')
# print(f'r^2 score: {r2_score(y_train, y_pred):.2f}')

# x_train.head()

In [119]:
gb_pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer()
    # XGBRegressor(n_estimators=200, objective='reg:squarederror', n_jobs=-1)
)

# gb_pipeline.fit(x_train, y_train)
# y_pred = gb_pipeline.predict(x_val) # any difference between this method, and the method of using early_stopping_rounds (below)?

x_train_processed = gb_pipeline.fit_transform(x_train) # transforming by going through pipeline
x_val_processed = gb_pipeline.transform(x_val)

eval_set = [(x_train_processed, y_train), 
            (x_val_processed, y_val)] # look at lecture (for explanation of this)

model = XGBRegressor(n_estimators=1000, n_jobs=-1, random_state=10)
model.fit(x_train_processed, y_train, eval_set=eval_set, eval_metric='mae', 
          early_stopping_rounds=30)

[09:32:40] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:127.361	validation_1-mae:125.952
Multiple eval metrics have been passed: 'validation_1-mae' will be used for early stopping.

Will train until validation_1-mae hasn't improved in 30 rounds.
[1]	validation_0-mae:114.675	validation_1-mae:113.784
[2]	validation_0-mae:103.348	validation_1-mae:103.146


/Users/diego/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/diego/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[3]	validation_0-mae:93.2229	validation_1-mae:93.463
[4]	validation_0-mae:84.359	validation_1-mae:85.1634
[5]	validation_0-mae:76.6704	validation_1-mae:77.8808
[6]	validation_0-mae:70.1	validation_1-mae:71.6809
[7]	validation_0-mae:64.4998	validation_1-mae:66.5639
[8]	validation_0-mae:59.5122	validation_1-mae:62.0886
[9]	validation_0-mae:55.5506	validation_1-mae:58.6149
[10]	validation_0-mae:52.2822	validation_1-mae:55.7873
[11]	validation_0-mae:49.5917	validation_1-mae:53.493
[12]	validation_0-mae:47.3743	validation_1-mae:51.7042
[13]	validation_0-mae:45.5143	validation_1-mae:50.228
[14]	validation_0-mae:43.974	validation_1-mae:49.0219
[15]	validation_0-mae:42.6939	validation_1-mae:48.0339
[16]	validation_0-mae:41.8499	validation_1-mae:47.3629
[17]	validation_0-mae:41.039	validation_1-mae:46.7736
[18]	validation_0-mae:40.3372	validation_1-mae:46.2483
[19]	validation_0-mae:39.8562	validation_1-mae:45.9555
[20]	validation_0-mae:39.392	validation_1-mae:45.6246
[21]	validation_0-mae:39.13

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=-1, nthread=None, objective='reg:linear', random_state=10,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [120]:
y_pred = model.predict(x_val_processed)

print('Gradient Boosting R^2', r2_score(y_val, y_pred))
print('Gradient Boosting MAE', mean_absolute_error(y_val, y_pred))
print('Gradient Boosting MSE', mean_squared_error(y_val, y_pred))

Gradient Boosting R^2 0.4803359147150632
Gradient Boosting MAE 41.56965401259111
Gradient Boosting MSE 14943.309528884718


In [121]:
x_test_processed = gb_pipeline.transform(x_test)
y_pred_test = model.predict(x_test_processed)

print('Gradient Boosting R^2', r2_score(y_pred_test, y_pred))
print('Gradient Boosting MAE', mean_absolute_error(y_pred_test, y_pred))
print('Gradient Boosting MSE', mean_squared_error(y_pred_test, y_pred))

Gradient Boosting R^2 -1.1299032874259995
Gradient Boosting MAE 83.74363
Gradient Boosting MSE 16597.04


In [122]:
# using the dropping duplicate columns AFTER I merge them instead of
# using the filter beforehand, gave me slighly worse mse and mae score

### Modeling for df_3

In [124]:
# now on new_df_3_sorted_cop

In [125]:
target = 'price'
features = new_train_df.drop(columns=target).columns.tolist()

# only when using train test split func
# X = new_train.drop(columns=target)
# y = new_train[target]

x_train_3 = new_train_df[features]
y_train_3 = new_train_df[target]

x_val_3 = val_new_df[features]
y_val_3 = val_new_df[target]

x_test_3 = test_new_df[features]
y_test_3 = test_new_df[target]

In [126]:
gb_pipeline_other = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer()
    # XGBRegressor(n_estimators=200, objective='reg:squarederror', n_jobs=-1)
)

# gb_pipeline.fit(x_train, y_train)
# y_pred = gb_pipeline.predict(x_val) # any difference between this method, and the method of using early_stopping_rounds (below)?

x_train_processed_3 = gb_pipeline_other.fit_transform(x_train_3) # transforming by going through pipeline
x_val_processed_3 = gb_pipeline_other.transform(x_val_3)

eval_set = [(x_train_processed_3, y_train_3), 
            (x_val_processed_3, y_val_3)] # look at lecture (for explanation of this)

model_3 = XGBRegressor(n_estimators=1000, n_jobs=-1, random_state=10)
model_3.fit(x_train_processed_3, y_train_3, eval_set=eval_set, eval_metric='mae', 
          early_stopping_rounds=30)

[09:32:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:127.354	validation_1-mae:125.28
Multiple eval metrics have been passed: 'validation_1-mae' will be used for early stopping.

Will train until validation_1-mae hasn't improved in 30 rounds.
[1]	validation_0-mae:114.681	validation_1-mae:113.377
[2]	validation_0-mae:103.31	validation_1-mae:102.431


/Users/diego/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/diego/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[3]	validation_0-mae:93.1822	validation_1-mae:92.9231
[4]	validation_0-mae:84.2117	validation_1-mae:84.3273
[5]	validation_0-mae:76.3823	validation_1-mae:77.0311
[6]	validation_0-mae:69.624	validation_1-mae:70.7911
[7]	validation_0-mae:63.9022	validation_1-mae:65.5346
[8]	validation_0-mae:59.0667	validation_1-mae:61.3451
[9]	validation_0-mae:55.0107	validation_1-mae:57.7158
[10]	validation_0-mae:51.6653	validation_1-mae:54.8617
[11]	validation_0-mae:48.9527	validation_1-mae:52.6384
[12]	validation_0-mae:46.7502	validation_1-mae:50.8422
[13]	validation_0-mae:44.9283	validation_1-mae:49.4219
[14]	validation_0-mae:43.5127	validation_1-mae:48.3497
[15]	validation_0-mae:42.4293	validation_1-mae:47.5419
[16]	validation_0-mae:41.5637	validation_1-mae:46.9524
[17]	validation_0-mae:40.5988	validation_1-mae:46.2496
[18]	validation_0-mae:40.0565	validation_1-mae:45.8485
[19]	validation_0-mae:39.4894	validation_1-mae:45.4623
[20]	validation_0-mae:39.1108	validation_1-mae:45.2953
[21]	validation_0-

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=-1, nthread=None, objective='reg:linear', random_state=10,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [127]:
y_pred_3 = model_3.predict(x_val_processed_3)

print('Gradient Boosting R^2', r2_score(y_val_3, y_pred_3))
print('Gradient Boosting MAE', mean_absolute_error(y_val_3, y_pred_3))
print('Gradient Boosting MSE', mean_squared_error(y_val_3, y_pred_3))

Gradient Boosting R^2 0.48942646989932326
Gradient Boosting MAE 40.544064431463134
Gradient Boosting MSE 14661.714108605636


In [128]:
x_test_processed_3 = gb_pipeline_other.transform(x_test_3)
y_pred_test_3 = model_3.predict(x_test_processed_3)

print('Gradient Boosting R^2', r2_score(y_pred_test_3, y_pred_test_3))
print('Gradient Boosting MAE', mean_absolute_error(y_pred_test_3, y_pred_3))
print('Gradient Boosting MSE', mean_squared_error(y_pred_test_3, y_pred_3))

Gradient Boosting R^2 1.0
Gradient Boosting MAE 84.94341
Gradient Boosting MSE 16690.826


In [129]:
# new_practice permutation importance

In [130]:
import eli5
from eli5.sklearn import PermutationImportance

permuter = PermutationImportance(
    model, 
    scoring='neg_mean_absolute_error',
    n_iter=5,
    random_state=10
)

permuter.fit(x_val_processed, y_val)

PermutationImportance(cv='prefit',
                      estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                             colsample_bylevel=1,
                                             colsample_bynode=1,
                                             colsample_bytree=1, gamma=0,
                                             importance_type='gain',
                                             learning_rate=0.1,
                                             max_delta_step=0, max_depth=3,
                                             min_child_weight=1, missing=None,
                                             n_estimators=1000, n_jobs=-1,
                                             nthread=None,
                                             objective='reg:linear',
                                             random_state=10, reg_alpha=0,
                                             reg_lambda=1, scale_pos_weight=1,
                                    

In [131]:
feature_names = x_val.columns.tolist()

pd.Series(permuter.feature_importances_, feature_names).sort_values(ascending=False)
eli5.show_weights(
    permuter, 
    top=None, # show permutation importances for all features
    feature_names=feature_names
)

Weight,Feature
14.0884 ± 0.6304,room_type_y
8.0984 ± 0.3178,longitude_y
6.4962 ± 0.2629,accommodates
4.7373 ± 0.3947,latitude_y
4.3780 ± 0.5750,bedrooms
2.3543 ± 0.3012,bathrooms
2.1360 ± 0.3685,availability_30
1.8090 ± 0.5806,neighbourhood_group
0.6549 ± 0.1256,availability_365
0.6487 ± 0.1228,is_business_travel_ready


In [ ]:

# new_df3 permutation importance

In [134]:
import eli5
from eli5.sklearn import PermutationImportance

permuter_3 = PermutationImportance(
    model_3, 
    scoring='neg_mean_absolute_error',
    n_iter=5,
    random_state=10
)

permuter_3.fit(x_val_processed_3, y_val_3)

PermutationImportance(cv='prefit',
                      estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                             colsample_bylevel=1,
                                             colsample_bynode=1,
                                             colsample_bytree=1, gamma=0,
                                             importance_type='gain',
                                             learning_rate=0.1,
                                             max_delta_step=0, max_depth=3,
                                             min_child_weight=1, missing=None,
                                             n_estimators=1000, n_jobs=-1,
                                             nthread=None,
                                             objective='reg:linear',
                                             random_state=10, reg_alpha=0,
                                             reg_lambda=1, scale_pos_weight=1,
                                    

In [135]:
feature_names_3 = x_val_3.columns.tolist()

pd.Series(permuter_3.feature_importances_, feature_names_3).sort_values(ascending=False)
eli5.show_weights(
    permuter_3, 
    top=None, # show permutation importances for all features
    feature_names=feature_names_3
)

Weight,Feature
17.5416 ± 0.5551,room_type
10.2975 ± 0.6335,longitude
7.1486 ± 0.7005,latitude
6.5342 ± 0.8206,accommodates
3.4842 ± 0.0548,bedrooms
3.2067 ± 0.2881,bathrooms
2.1925 ± 0.2727,availability_30
0.8143 ± 0.0994,amenities
0.6598 ± 0.2226,availability_365
0.6370 ± 0.0585,guests_included
